# About this notebook

This notebook assumes you have ran the local Census Regression notebook and you have not deleted the LOCAL_ROOT folder. In this notebook, we will deploy a trained model and submit prediction requests.

# Setting things up

In [1]:
import mltoolbox.regression.dnn as sd

In [2]:
import os
import pandas as pd
import tensorflow as tf
from tensorflow.python.lib.io import file_io

This notebook will write files during prediction. Please give a root folder you wish to use.

In [3]:
LOCAL_ROOT = './census_regression_workspace' # This should be the same as what was used in the local census notebook
CLOUD_ROOT = 'gs://' + datalab_project_id() + '-census-regression-datalab'

# No need to edit anything else in this cell.
LOCAL_TRAINING_DIR = os.path.join(LOCAL_ROOT, 'training')
CLOUD_TRAINING_DIR = os.path.join(CLOUD_ROOT, 'training_deploy')

if not file_io.file_exists(LOCAL_ROOT):
  raise ValueError('LOCAL_ROOT not found. Did you run the local notebook?')
  
!gsutil mb {CLOUD_ROOT}

Creating gs://cloud-ml-dev-census-regression-datalab/...
ServiceException: 409 Bucket cloud-ml-dev-census-regression-datalab already exists.


First, let us put the csv files on GCS and the output of training.

In [4]:
!gsutil -m rm -fr {CLOUD_TRAINING_DIR}

CommandException: 1 files/objects could not be removed.


In [5]:
!gsutil -m cp -r {LOCAL_TRAINING_DIR} {CLOUD_TRAINING_DIR}

Copying file://./census_regression_workspace/training/features_file.json [Content-Type=application/json]...
Copying file://./census_regression_workspace/training/model/saved_model.pb [Content-Type=application/octet-stream]...
Copying file://./census_regression_workspace/training/model/variables/variables.index [Content-Type=application/octet-stream]...
Copying file://./census_regression_workspace/training/model/variables/variables.data-00000-of-00001 [Content-Type=application/octet-stream]...
Copying file://./census_regression_workspace/training/model/assets.extra/schema.json [Content-Type=application/json]...
Copying file://./census_regression_workspace/training/evaluation_model/saved_model.pb [Content-Type=application/octet-stream]...
Copying file://./census_regression_workspace/training/train/graph.pbtxt [Content-Type=application/octet-stream]...
Copying file://./census_regression_workspace/training/model/assets.extra/features.json [Content-Type=application/json]...
Copying file://.

In [6]:
!gsutil ls {CLOUD_TRAINING_DIR}

gs://cloud-ml-dev-census-regression-datalab/training_deploy/features_file.json
gs://cloud-ml-dev-census-regression-datalab/training_deploy/evaluation_model/
gs://cloud-ml-dev-census-regression-datalab/training_deploy/model/
gs://cloud-ml-dev-census-regression-datalab/training_deploy/train/


# Online Prediction using ML Engine

We first have to deploy the model, and then we can make requests on it.

In [7]:
!gcloud ml-engine models create census_model --regions=us-central1
!gcloud ml-engine versions create v1 --model census_model --origin {CLOUD_TRAINING_DIR}/model

Creating version (this might take a few minutes)......done.


In [8]:
sd.predict(
  cloud=True,
  model_name='census_model',
  model_version='v1',
  data=['490,64,2,0,1,0,2,8090,015,01,1,00590,00500,1,18,0,2,1',
        '1225,32,5,0,4,5301,2,9680,015,01,1,00100,00100,1,21,2,1,1',
        '1226,30,1,0,1,0,2,8680,020,01,1,00100,00100,1,16,0,2,1']
)

,SERIALNO,predicted_target
0,490,30.34
1,1225,64.8666
2,1226,19.3237


In [9]:
sd.predict(
  cloud=True,
  model_name='census_model',
  model_version='v1',
  data=pd.DataFrame(
    [[490,64,2,0,1,0,2,8090,"015","01",1,"00590","00500",1,18,0,2,1],
     [1225,32,5,0,4,5301,2,9680,"015","01",1,"00100","00100",1,21,2,1,1],
     [1226,30,1,0,1,0,2,8680,"020","01",1,"00100","00100",1,16,0,2,1]])
)

,SERIALNO,predicted_target
0,490,30.34
1,1225,64.8666
2,1226,19.3237


# Cleaning things up

If you want to delete the files you made on GCS, uncomment and run the next cell.

In [10]:
#!gsutil rm -fr {CLOUD_ROOT}

The next cell deletes the deployed model. Don't run it if you want to keep it around.

In [11]:
!gcloud ml-engine versions delete v1 --model census_model 
!gcloud ml-engine models delete census_model 

Deleting version [v1]......done.
Deleting model [census_model]...done.
